In [1]:
from __future__ import print_function

import datetime
import numpy as np
import glob
import warnings

from multiprocessing import Pool

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, get_full_id_from_partial_id
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status



/Users/egentry/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/egentry/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:41: UserWarning: `session' from database_helpers can only write using 1 process at a time
  UserWarning)


#### Helpful resource:
http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [2]:
data_dir = "../saved_runs/cluster_parameter_study/"

In [3]:
def parse_into_simulations(data_dir_and_id):
    # wrapper function to allow multiprocessing + map
    data_dir, id = data_dir_and_id
    run_summary = RunSummary(data_dir=data_dir, id=id)

    simulation = Simulation.from_run_summary(run_summary)
    
    inputs = Inputs(os.path.join(data_dir, id + "_inputs.dat"))
    simulation_inputs = Simulation_Inputs.from_Inputs(id, inputs)
    
    simulation_status = Simulation_Status.from_run_summary(run_summary)
    
    
    return simulation, simulation_inputs, simulation_status
        

# Check which rows already exist

In [4]:
for sim in session.query(Simulation):
    print(sim)

<Simulation: 000c7c73-33da-42e3-9b87-d2e61a386d08, last updated: 2016-01-06 17:40:22.533656
<Simulation: 016530ff-83c1-4200-9b95-96a06c1a3957, last updated: 2016-01-06 17:40:36.248599
<Simulation: 05219c9a-2fb3-4943-813f-5dea43c1f822, last updated: 2016-01-06 17:41:16.841933
<Simulation: 069a31c0-ff00-4a69-a320-f734877d732b, last updated: 2016-01-06 17:41:19.281474
<Simulation: 0716f4d7-9411-40d3-b001-4a021cf0c2e1, last updated: 2016-01-06 17:41:25.498389
<Simulation: 073b72de-970e-4f09-96dd-708e27688e95, last updated: 2016-01-06 17:41:41.431658
<Simulation: 0ba8b023-c555-45c9-8fb2-dd7a4c6da1e6, last updated: 2016-01-06 17:42:28.747060
<Simulation: 0ca1a191-8aae-4577-a233-7926a6ab7e02, last updated: 2016-01-06 17:42:40.804689
<Simulation: 0e13dae0-7ed9-4798-a158-63e9cba63636, last updated: 2016-01-06 17:43:04.075339
<Simulation: 0e558316-efa9-4259-a48c-eea876c7d618, last updated: 2016-01-06 17:43:12.722461
<Simulation: 0edcca3b-6d7f-45ea-9dc2-a7172d68a01d, last updated: 2016-01-06 17:4

In [5]:
for sim_inputs in session.query(Simulation_Inputs):
    print(sim_inputs)

<Simulation_Inputs for id: 000c7c73-33da-42e3-9b87-d2e61a386d08>
<Simulation_Inputs for id: 016530ff-83c1-4200-9b95-96a06c1a3957>
<Simulation_Inputs for id: 05219c9a-2fb3-4943-813f-5dea43c1f822>
<Simulation_Inputs for id: 069a31c0-ff00-4a69-a320-f734877d732b>
<Simulation_Inputs for id: 0716f4d7-9411-40d3-b001-4a021cf0c2e1>
<Simulation_Inputs for id: 073b72de-970e-4f09-96dd-708e27688e95>
<Simulation_Inputs for id: 0ba8b023-c555-45c9-8fb2-dd7a4c6da1e6>
<Simulation_Inputs for id: 0ca1a191-8aae-4577-a233-7926a6ab7e02>
<Simulation_Inputs for id: 0e13dae0-7ed9-4798-a158-63e9cba63636>
<Simulation_Inputs for id: 0e558316-efa9-4259-a48c-eea876c7d618>
<Simulation_Inputs for id: 0edcca3b-6d7f-45ea-9dc2-a7172d68a01d>
<Simulation_Inputs for id: 0fdca3cb-3c60-4073-b3d5-33b771786bfc>
<Simulation_Inputs for id: 1123c381-e88f-4e42-adbc-95cfa2b21daa>
<Simulation_Inputs for id: 130ce300-e97e-47e8-a8c9-b6d56126c2bb>
<Simulation_Inputs for id: 16f7d335-77b9-4ad9-8eba-153c57026ed8>
<Simulation_Inputs for id

# Add files to database in batch

In [15]:
id = "26f97021-7ea1-4db7-bcc2-a7899841ebac"

s = session.query(Simulation_Status).get(id)
s.status = "Running"

In [6]:
def add_entire_dir_to_database(data_dir):
    overview_filenames = glob.glob(os.path.join(data_dir, "*overview.dat"))
    ids = [None]*len(overview_filenames)
    for k, filename in enumerate(overview_filenames):
        ids[k] = os.path.basename(filename).split("_overview.dat")[0]

    mappable_inputs = [(data_dir, id) for id in ids]
    
    pool = Pool(4)
    simulation_tuples = pool.map(parse_into_simulations, mappable_inputs)
    for simulation, simulation_inputs, simulation_status in simulation_tuples:
        simulation.add_or_update_to_table()
        simulation_inputs.add_or_update_to_table()
        simulation_status.add_or_update_to_table()
add_entire_dir_to_database(data_dir)

unreasonable jump before checkpoint: ../saved_runs/cluster_parameter_study/26f97021-7ea1-4db7-bcc2-a7899841ebac_checkpoint_0046.dat
unreasonable jump before checkpoint: ../saved_runs/cluster_parameter_study/417eceab-0a0a-4040-a087-e33d4c38f7dc_checkpoint_0029.dat
unreasonable jump before checkpoint: ../saved_runs/cluster_parameter_study/f8644401-fcff-4a77-bf47-2418e60d19ae_checkpoint_0070.dat
unreasonable jump before checkpoint: ../saved_runs/cluster_parameter_study/d431dd27-ee94-4f72-b81d-e572f9f74266_checkpoint_0044.dat


/Users/egentry/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:301: UserWarning: Strange momenta extrema after last SNe; id: 01b35950-99b7-49cd-8a85-e8ad5fbafa70
  warnings.warn("Strange momenta extrema after last SNe; id: " + id, UserWarning)
/Users/egentry/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:301: UserWarning: Strange momenta extrema after last SNe; id: 26887be5-61d4-44f8-8119-698c4344c560
  warnings.warn("Strange momenta extrema after last SNe; id: " + id, UserWarning)
/Users/egentry/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:301: UserWarning: Strange momenta extrema after last SNe; id: 3afa3247-4118-4a2d-8c49-6179e9bb98c0
  warnings.warn("Strange momenta extrema after last SNe; id: " + id, UserWarning)
/Users/egentry/Dropbox/data/clustered_SNe/analysis/../../clustered_SNe/analysis/database_helpers.py:301: UserWarning: Strange momenta extrema after last SNe;

# Check statuses from database

In [31]:
statuses = np.array([status.status for status in session.query(Simulation_Status)])
counts = {status : 0 for status in Simulation_Status.possible_statuses}

for status in statuses:
    counts[status] += 1

print(counts)

{'Ready': 29, 'Complete': 179, 'Error': 0, 'Unknown': 0, 'Running': 37}


In [17]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Ready"):
    print(simulation_status.id)

16f7d335-77b9-4ad9-8eba-153c57026ed8
179c5f0b-ce52-4f81-84b2-b6afb47cb335
23b494ec-5852-4409-b7dd-430dc3f8446a
296f7e76-6ea0-44d9-9ef7-b826ef47ea36
35991ff8-2185-4b73-bc31-5ab852e07387
39758925-5bcf-475e-8f44-1e46e44cde75
3afa3247-4118-4a2d-8c49-6179e9bb98c0
3b381194-fe9b-4f6b-afc6-33e641f698c1
4ab2cbde-90cd-4109-8754-6ef7f5a79898
6fbde823-0019-4948-b629-2dded4fad53e
7e1d38da-82b4-42b0-a351-f59191d79995
8074e5f2-7c97-4a21-ae77-797c942209e4
a0b86f69-f212-4c3a-99ad-e75245dd5263
a9d669ef-1ccd-47db-ad2b-c500ec9de80d
ae1f1577-ad61-4f26-99ef-1204be73b417
caf6ac95-8d17-4f2d-957b-bcaff6f1d370
d5bd1990-1c43-407f-8abd-770dcbcf3a53
e416222b-91df-4bc9-9f0b-7e8c9911ba05
e4f585ee-310d-4c90-9b68-8a6a5560385b
fd7ec1de-db4e-4519-90a8-9f638bd54cf7
740651bb-3a82-4b6d-96b0-3811dc3f91b8
929cdeab-2750-4289-ac04-1fa319606590
9c6d8e76-7923-4702-b6e9-f88baa0c8209
e19f5830-0c85-47c8-9f02-a657fcde7a4b


In [30]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Error"):
    print(simulation_status.id)

In [19]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Running"):
    print(simulation_status.id)

0716f4d7-9411-40d3-b001-4a021cf0c2e1
0edcca3b-6d7f-45ea-9dc2-a7172d68a01d
1123c381-e88f-4e42-adbc-95cfa2b21daa
1cf82e21-095e-4eb6-b554-60a5d9363762
26f97021-7ea1-4db7-bcc2-a7899841ebac
2ec6b483-da85-4781-b5f0-075531580754
417eceab-0a0a-4040-a087-e33d4c38f7dc
45e9c742-7d2c-461b-ad63-5e1be00777c9
560f1259-63e6-40cf-b5ce-40b715490d5c
56d7e5b5-3845-441a-83fa-2996f493501c
572d20e4-89e5-48b9-983b-86f8833c2570
5ac6f05a-b4f4-4550-aac9-c6e5327e9f32
5d571036-8dab-4e3f-923d-d058d88f7e96
60dc824d-d348-425e-8972-a1e84c8ae663
6d4a4a30-4153-47b7-94ac-fb3055e8dba8
7dc91bd8-ba64-4148-b560-ebe6b8c1c265
9ce53082-875a-42e7-b938-e57926ddb673
a302e259-80cd-425e-b0b0-278ab8778c88
a3f38dec-7696-44cb-b376-7462997d9a97
a4b88f80-50f9-496a-8b0b-8234b05e4dc0
a7bc1f23-aff9-4dcf-9705-03ae1f8e2507
c668b684-b0ac-474f-bdc0-a985c1b695b7
c7b202a2-dbf2-46fb-a192-4ab258b70b9d
cbd8bbe5-389f-438e-b324-7bf3ecc1b46d
d173f8fd-ae0b-4809-895f-6b19e1f11941
d431dd27-ee94-4f72-b81d-e572f9f74266
d45799c7-f947-4945-a247-8fab1e1d8aa7
D

In [26]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Unknown"):
    print(simulation_status.id)

In [29]:
id = "f2f6e09f-dd2b-4d98-80fa-1e2f63ed8a9a"

s = session.query(Simulation_Status).get(id)
s.status = "Running"

# Restart if ready

In [ ]:
files = glob.glob("../scripts/SGE/tmp_restart/*")
for f in files:
    os.remove(f)
    
f_submit = open("submit_all", mode="w")
f_submit.write("#!/bin/bash \n")
for status in session.query(Simulation_Status).filter(Simulation_Status.status == "Ready"):
    print(status.id)
    status.create_new_batch_file_and_switch_to_running()
    f_submit.write("qsub ../scripts/SGE/restart.batch." + status.id + "\n")
    
f_submit.close()

# Shut down SQL session when done

In [ ]:
session.commit()

In [ ]:
session.close()